In [1]:
import numpy as np
import tensorflow as tf

In [ ]:
def nn(x,initvalue):
    return tf.reduce_sum(x)+initvalue

T=8
tf.reset_default_graph()
g=tf.get_default_graph()


source=tf.constant([1,2,3,4,5,6,7,8],dtype=tf.float32)
X=tf.get_variable('X',shape=[2])
t=tf.placeholder(dtype=tf.int32)
x_update_op=tf.assign(X,source[t*2:t*2+2])

with g.control_dependencies([x_update_op]):
    init_state=tf.get_variable('init_state',shape=[],initializer=tf.zeros_initializer)
    final_state=nn(X,init_state)
    update_op=tf.assign(init_state,final_state)
    op2=tf.group([update_op])

sess=tf.Session()
sess.run(tf.global_variables_initializer())
for _t in range(4):
#     print(sess.run(update_op,feed_dict={t:_t}))
    print(sess.run(update_op,feed_dict={t:_t}))


In [ ]:
def randomFile(path,N,T,D):
    with open(path,'w') as f:
        for n in range(N):
            s=''
            for t in range(T):
                s1=''
                for d in range(D):
                    s1+=str(n+d)+','
                s+=s1
            s=s[:-1]
            
            f.write(s+'\n')
N,T,D=200,30,3
randomFile('/home/zhangxk/my.txt',N,T,D)

In [29]:
def data_pipe(datafile,batch_size,T,D):
    '''
        返回一个(batch_size,T,D)的source,表示每一次从文件取出来的一批数据
        iterator:迭代对象,用户初始化数据源
    '''
    dataset=tf.data.TextLineDataset(datafile)
    #每一行用,分割开
    dataset=dataset.map(lambda line:tf.string_split([line],',').values)
    #字符串数组转成float32
    dataset=dataset.map(lambda x:tf.string_to_number(x,tf.float32))
    #数据分割成batch
    dataset=dataset.batch(batch_size)
    
    iterator = dataset.make_initializable_iterator()
    #把dataset转成tensor,shape[batch,?]
    source_batch=iterator.get_next()
    source_batch=tf.reshape(source_batch,shape=[-1,T,D])
    return source_batch,iterator
def prepareInput(source,period,batch,T,D):
    '''
    source是一个很长的序列,shape[?,T,D],
    创建RNN的一个输入X,shape[batch,period,D],使用一个游标cursor,
    每次把source中period序列取出,赋值给X
    
    返回 :
    X:RNN的输入
    assign_op:
    cursor:一个placeholder,游标
    
    
    '''
    #创建一个source的副本,然后把source复制给tmp_source
    tmp_source=tf.get_variable('tmp_source',shape=[batch,T,D],trainable=False)
    tmp_source_op=tf.assign(tmp_source,source)
    
    #真实的source,batchsize不一定是batch,所有在刷新数据源的时候要保存实际batch长度
    real_batch=tf.get_variable('real_batch',shape=[],dtype=tf.int32,trainable=False)
    REAL_BATCH=tf.shape(source)[0]
    real_batch_assign_op=tf.assign(real_batch,REAL_BATCH)
    
    cursor=tf.placeholder(tf.int32)
    X=tf.get_variable('input',shape=[batch,period,D],trainable=False)
    X=tf.assign(X,tmp_source[:,cursor*period:cursor*period+period])
    
    op=tf.group([tmp_source_op,real_batch_assign_op])
    return X[:real_batch],op,cursor

In [30]:
N,T,D=200,30,3
period=2
batch=1

tf.reset_default_graph()
source,source_iterator=data_pipe('/home/zhangxk/my.txt',batch,T,D)
X,assignSource,cursor=prepareInput(source,period,batch,T,D)

In [32]:
sess=tf.Session()
sess.run(tf.global_variables_initializer())
sess.run(source_iterator.initializer)

sess.run(assignSource)
for i in range(10):
    for t in range(T//period):
        print(sess.run(X,feed_dict={cursor:t}))
    sess.run(assignSource)

[[[0. 1. 2.]
  [0. 1. 2.]]]
[[[0. 1. 2.]
  [0. 1. 2.]]]
[[[0. 1. 2.]
  [0. 1. 2.]]]
[[[0. 1. 2.]
  [0. 1. 2.]]]
[[[0. 1. 2.]
  [0. 1. 2.]]]
[[[0. 1. 2.]
  [0. 1. 2.]]]
[[[0. 1. 2.]
  [0. 1. 2.]]]
[[[0. 1. 2.]
  [0. 1. 2.]]]
[[[0. 1. 2.]
  [0. 1. 2.]]]
[[[0. 1. 2.]
  [0. 1. 2.]]]
[[[0. 1. 2.]
  [0. 1. 2.]]]
[[[0. 1. 2.]
  [0. 1. 2.]]]
[[[0. 1. 2.]
  [0. 1. 2.]]]
[[[0. 1. 2.]
  [0. 1. 2.]]]
[[[0. 1. 2.]
  [0. 1. 2.]]]
[[[1. 2. 3.]
  [1. 2. 3.]]]
[[[1. 2. 3.]
  [1. 2. 3.]]]
[[[1. 2. 3.]
  [1. 2. 3.]]]
[[[1. 2. 3.]
  [1. 2. 3.]]]
[[[1. 2. 3.]
  [1. 2. 3.]]]
[[[1. 2. 3.]
  [1. 2. 3.]]]
[[[1. 2. 3.]
  [1. 2. 3.]]]
[[[1. 2. 3.]
  [1. 2. 3.]]]
[[[1. 2. 3.]
  [1. 2. 3.]]]
[[[1. 2. 3.]
  [1. 2. 3.]]]
[[[1. 2. 3.]
  [1. 2. 3.]]]
[[[1. 2. 3.]
  [1. 2. 3.]]]
[[[1. 2. 3.]
  [1. 2. 3.]]]
[[[1. 2. 3.]
  [1. 2. 3.]]]
[[[1. 2. 3.]
  [1. 2. 3.]]]
[[[2. 3. 4.]
  [2. 3. 4.]]]
[[[2. 3. 4.]
  [2. 3. 4.]]]
[[[2. 3. 4.]
  [2. 3. 4.]]]
[[[2. 3. 4.]
  [2. 3. 4.]]]
[[[2. 3. 4.]
  [2. 3. 4.]]]
[[[2. 3. 4.]
  [2. 3

In [ ]:
2000//128

In [ ]:
tf.reset_default_graph()
source=tf.random_uniform(shape=[2])
X=tf.get_variable('X',shape=[2])
assign_X=tf.assign(X,source)
sess=tf.Session()
sess.run(tf.global_variables_initializer())
for i in range(3):
    print(sess.run(assign_X))
    print(sess.run(X))

In [ ]:
def valid_unit(cursor,times):
    tf.whr